In [1]:
import numpy as np
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from models.detr import DETR
from models.segmentation import DETRsegm
from hubconf import detr_resnet101_panoptic, detr_resnet101
from torchvision.transforms import Resize

In [2]:
from models.hub.resnet import _resnet

In [3]:
# class DETRsegm_3D(nn.Module):
#     def __init__(self):
#         detr = detr_resnet101_panoptic()
#         conv

In [4]:
def sort_func(path):
    path_id = int(path.split('/')[-1].split('_')[1])
    return path_id

In [5]:
def show_image_and_label(image, label):
    fig, axs = plt.subplots(nrows=1,ncols=2, squeeze=False,figsize=(12, 12))
    axs[0, 0].imshow(image)
    axs[0, 0].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Image')
    axs[0, 1].imshow(label)
    axs[0, 1].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[], title='Label')

In [6]:
class DatasetForSegmentation(Dataset):
    
    def __init__(self, image_paths, label_paths):
        self.image_paths = image_paths
        self.label_paths = label_paths
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, i):
        image_np_array = nib.load(image_paths[i]).get_fdata()
        image_torch_tensor = torch.from_numpy(image_np_array)
        label_np_array = nib.load(image_paths[i]).get_fdata()
        label_torch_tensor = torch.from_numpy(label_np_array)
        return image_torch_tensor, label_np_array

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
# path = '/home/francisco/workspace/ImageCHD_dataset'

In [9]:
# image_paths = glob(f'{path}/*image.nii.gz',recursive=True)
# label_paths = glob(f'{path}/*label.nii.gz',recursive=True)

In [10]:
# image_paths.sort(key=sort_func)
# label_paths.sort(key=sort_func)

In [11]:
# dset = DatasetForSegmentation(image_paths,label_paths)

In [12]:
# inpt, outp = dset[2]

In [13]:
# model = _resnet();
# model = model.half()
# model.to(device);

In [14]:
# inpt_right_shape = inpt.transpose(2,0).unsqueeze(0).unsqueeze(0).float()
# new_size = [int(inpt_right_shape.shape[2]/3),int(inpt_right_shape.shape[3]/3),int(inpt_right_shape.shape[4]/3)]
# inpt_resized = inpt_right_shape.resize_((1,1,new_size[0],new_size[1],new_size[2]))

In [15]:
# cv_3d = torch.nn.Conv3d(1,3,(1,3,3),padding = 1)
# result = cv_3d(inpt_resized)
# result_cuda = result.to(device)
# result_2 = model(result_cuda.half())

In [16]:
# detr = detr_resnet101_panoptic();
# detr.eval();

***

In [20]:
detr_seg = detr_resnet101_panoptic()
detr_seg.eval();
detr_seg.to(device);
resnet_3d = _resnet();
resnet_3d.to(device);
detr_seg.detr.backbone[0] = resnet_3d

In [21]:
im = torch.ones((1, 1, 32, 64, 64),device=device)
# im = torch.ones((1, 3, 128, 128),device=device)

In [22]:
preds = detr_seg(im)

/home/francisco/workspace/CHD_Classifier_by_Francisco_Lourenço/models/position_encoding.py:84: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


torch.Size([1, 2048, 8, 16, 16])



IndexError: list index out of range

3D: torch.Size([1, 2048, 8, 16, 16])

2D: torch.Size([1, 256, 32, 32]) torch.Size([1, 512, 16, 16]) torch.Size([1, 1024, 8, 8]) torch.Size([1, 2048, 4, 4])

In [ ]:
preds

In [ ]:
preds['pred_logits'].shape, preds['pred_boxes'].shape

***

***

In [ ]:
""" The forward expects a NestedTensor, which consists of:
       - samples.tensor: batched images, of shape [batch_size x 3 x H x W]
       - samples.mask: a binary mask of shape [batch_size x H x W], containing 1 on padded pixels

torch.Size([1, 1, 32, 128, 128]) torch.Size([1, 32, 128, 128])

torch.Size([1, 2048, 8, 32, 32]) torch.Size([1, 32, 128, 128])

torch.Size([1, 2048, 4, 4]) torch.Size([1, 4, 4])


torch.Size([1, 256, 4, 4])



torch.Size([1, 2048, 8, 32, 32]) torch.Size([1, 32, 128, 128])


torch.Size([1, 384, 32, 128, 128])

torch.Size([1, 256, 8, 32, 32]) torch.Size([1, 32, 128, 128]) torch.Size([100, 256]) torch.Size([1, 256, 32, 128, 128])

2D:

torch.Size([1, 256, 4, 4]) torch.Size([1, 4, 4]) torch.Size([100, 256]) torch.Size([1, 256, 4, 4])